In [1]:
## IMPORT LIBRARIES ----------------------------------------------------------------------------------------------------

%matplotlib qt

%reload_ext autoreload
%autoreload 3

import sys                                                                                                              # Import sys to add paths to libraries
import re                                                                                                               # Import re to work with regular expressions
import glob                                                                                                             # Import glob to read files
import matplotlib.pyplot as plt                                                                                         # Import matplotlib.pyplot to plot figures
import tkinter as tk                                                                                                    # Import TK to open folder dialogs to select folders
from tkinter import filedialog                                                                                          # Import filedialog to open folder dialogs to select folders
import numpy                                                                                                            # Import numpy to work with arrays and make calculations
from shutil import rmtree
import random                                                                                                           # Import random to make random choices
from datetime import datetime, timedelta                                                                                # Import time to measure time 
import time                                                                                                             # Import time to measure time
import os                                                                                                               # Import path to work with paths
import pandas                                                                                                           # Import pandas to work with dataframes
import warnings                                                                                                         # Import warnings to ignore warnings
warnings.filterwarnings('ignore')                                                                                       # Ignore warnings

## IMPORT CIRCADIPY ----------------------------------------------------------------------------------------------------

parent_path = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(parent_path)
import chrono_reader as chr                                                                                             # Import chrono_reader to read data
import chrono_plotter as chp                                                                                            # Import chrono_plotter to plot data
import chrono_rhythm as chrt                                                                                             # Import chrono_rithm to make calculations
import chrono_simulation as chs                                                                                         # Import chrono_simulation to simulate data


PyMICE library v. 1.2.1

The library is available under GPL3 license; we ask that reference to our paper
as well as to the library itself is provided in any published research making
use of PyMICE. Please run:

>>> print(pm.__REFERENCING__)

for more information (given that the library is imported as `pm`).




In [2]:
## SET THE ENVIRONMENT -------------------------------------------------------------------------------------------------

root = tk.Tk()                                                                                                          # Create the root window
root.attributes('-topmost',True)                                                                                        # Keep the window on top of others
root.iconify()                                                                                                          # Hide the root window

data_folder = filedialog.askdirectory(title='Select the folder containing the data')                                    # Ask the user to select the folder containing the data
root_folder = os.path.dirname(data_folder)                                                                              # Get the root folder
result_folder = os.path.join(root_folder, 'results_intellicage')                                                        # Create the result folder

if not os.path.exists(result_folder):                                                                                   # If the result folder does not exist
    os.makedirs(result_folder)                                                                                          # Create it
else:
    print('The folder already exists. The data will be overwritten.')                                                   # Otherwise, warn the user that the data will be overwritten

print('The data folder is: {}'.format(data_folder))                                                                     # Print the data folder
print('The result folder is: {}'.format(result_folder))                                                                 # Print the result folder

The folder already exists. The data will be overwritten.
The data folder is: C:/Users/morei/Documents/GitHub/circadipy/src/analysis_examples/intellicage/data_after_unwrapping
The result folder is: C:/Users/morei/Documents/GitHub/circadipy/src/analysis_examples/intellicage\results_intellicage


In [3]:
individual_files = glob.glob(data_folder + "/*.txt")
print(individual_files)

['C:/Users/morei/Documents/GitHub/circadipy/src/analysis_examples/intellicage/data_after_unwrapping\\data_animal_17.txt', 'C:/Users/morei/Documents/GitHub/circadipy/src/analysis_examples/intellicage/data_after_unwrapping\\data_animal_19.txt']


In [4]:
zt_0_time = 20
#labels_dict = {'cycle_types': ['LD', 'DD', 'DD', 'LD'], 'test_labels': ['1_control_dl', '2_control_dd', '3_hypo_dd', '4_hypo_dl'], 'cycle_days': [7, 29, 43, 13]}
labels_dict = {'cycle_types': ['LD', 'DD', 'DD', 'DD', 'LD'], 'test_labels': ['1_control_dl', '2_control_dd', '3_induction_dd', '4_hypo_dd', '5_hypo_dl'], 'cycle_days': [13, 29, 23, 20, 12]}
type = 'intellicage'

animals = {}
for file in individual_files:
    expression = r"\\data_(.*?)\.txt"
    name = re.search(expression, file)
    name = name.group(1)

    animals[name] = {}
    animals[name]['file'] = file

    protocol = chr.read_protocol(name, file, zt_0_time = zt_0_time, labels_dict = labels_dict, type = type, consider_first_day = False)
    protocol.resample('1H', method = 'sum')
    protocol.apply_filter(window = 3, type = 'moving_average', order = 2, reverse = False)
    protocol.normalize_data(type = 'minmax', per_day = True)
    protocol.delete_first_days(7)
    protocol.delete_last_days(2)
    protocol.delete_period(5,18, '2_control_dd')

    animals[name]['protocol'] = protocol

['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22', '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26', '2022-06-27', '2022-06-28', '2022-06-29', '2022-06-30', '2022-07-01', '2022-07-02']
['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22', '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26', '2022-06-27', '2022-06-28', '2022-06-29', '2022-06-30', '2022-07-01', '2022-07-02']


In [6]:
animals['animal_17']['protocol'].data

,values,is_night,cycle_types,test_labels,real_date,day
2022-06-10 00:00:00,0.64,False,LD,1_control_dl,2022-06-10 20:00:00,2022-06-10
2022-06-10 01:00:00,0.32,False,LD,1_control_dl,2022-06-10 21:00:00,2022-06-10
2022-06-10 02:00:00,0.12,False,LD,1_control_dl,2022-06-10 22:00:00,2022-06-10
2022-06-10 03:00:00,0.04,False,LD,1_control_dl,2022-06-10 23:00:00,2022-06-10
2022-06-10 04:00:00,0.00,False,LD,1_control_dl,2022-06-11 00:00:00,2022-06-10
...,...,...,...,...,...,...
2022-08-21 19:00:00,0.60,True,LD,5_hypo_dl,2022-09-05 15:00:00,2022-09-04
2022-08-21 20:00:00,0.40,True,LD,5_hypo_dl,2022-09-05 16:00:00,2022-09-04
2022-08-21 21:00:00,0.55,True,LD,5_hypo_dl,2022-09-05 17:00:00,2022-09-04
2022-08-21 22:00:00,0.90,True,LD,5_hypo_dl,2022-09-05 18:00:00,2022-09-04


In [7]:
reset = True

def reset_folder(folder):
    for root, dirs, files in os.walk(folder):
        for f in files:
            os.unlink(os.path.join(root, f))
        for d in dirs:
            rmtree(os.path.join(root, d))

if result_folder.split("\\")[-1] not in os.listdir(root_folder):
    os.mkdir(result_folder)
    print("Folder results created!")

for count, key in enumerate(list(animals.keys())):
    animals[key]['save_folder'] = result_folder + '\\' + key
    if key not in os.listdir(result_folder):
        os.mkdir(result_folder + '\\' + key)
        print("Folder " + key + " created!")
    elif key in os.listdir(result_folder) and reset == True:
        reset_folder(result_folder + '\\' + key)
        print("Folder " + key + " cleaned!")
    else:
        print("Folder " + key + " already exists!")

Folder animal_17 created!
Folder animal_19 created!


In [8]:
format = 'svg'

In [9]:
for animal in animals:
    chp.actogram_bar(animals[animal]['protocol'], first_hour = 18, save_folder = animals[animal]['save_folder'], save_suffix = 'bar', adjust_figure = [2, 0.85, 0.85, 0.2, 0.15], format = format, x_label = 'TIME (HOUR)', ticks_fontsize=[11,11], )
    chp.actogram_colormap(animals[animal]['protocol'], first_hour = 18, save_folder = animals[animal]['save_folder'], save_suffix = 'colormap', adjust_figure = [2, 0.85, 0.85, 0.2, 0.15], format = format, norm_color = None, x_label = 'TIME (HOUR)', ticks_fontsize=[11,11], unit_of_measurement='VISITS')
    chp.data_periodogram(animals[animal]['protocol'], time_shape = 'continuous', method = 'periodogram', max_period = 48, unit_of_measurement = 'VISITS', save_folder = animals[animal]['save_folder'], save_suffix = 'periodogram', format = format, ticks_fontsize=[14,14], labels_fontsize=[14,16,16])
    chp.data_periodogram(animals[animal]['protocol'], time_shape = 'continuous', method = 'welch', max_period = 48, unit_of_measurement = 'VISITS', save_folder = animals[animal]['save_folder'], save_suffix = 'welch', format = format, ticks_fontsize=[14,14], labels_fontsize=[14,16,16])

    print(animal.replace('_', ' ').capitalize() + ' actograms saved!')

Animal 17 actograms saved!
Animal 19 actograms saved!


In [10]:
for count, animal in enumerate(animals):
    init = time.time()

    save_folder = animals[animal]['save_folder']
    chp.time_serie(animals[animal]['protocol'], labels=['TIME SERIES', 'TIME (DAYS)', 'VISITS'],
                   color = 'midnightblue', save_folder = save_folder, save_suffix = '', format = format)
    chp.time_serie_sum_per_day(animals[animal]['protocol'], labels=['TIME SERIES', 'TIME (DAYS)', 'VISITS'], 
                               color = 'midnightblue', save_folder = save_folder, save_suffix = '', 
                               format = format)

    end = time.time() - init

    print(animal.replace('_', ' ').capitalize() + ' total activity per day saved! (' + str(round(end, 2)) + ' seconds)')
    

Animal 17 total activity per day saved! (0.62 seconds)
Animal 19 total activity per day saved! (0.46 seconds)


In [11]:
dict = {'time_shape': 'continuous', 
        'step': 0.01, 
        'start_time': 22, 
        'end_time': 26, 
        'n_components': [1]}

best_models = []

for count, animal in enumerate(animals):
    init = time.time()

    save_folder = animals[animal]['save_folder']

    best_models, best_models_file = chrt.fit_cosinor(animals[animal]['protocol'], dict = dict, save_folder = save_folder)
    best_models_fixed, best_models_fixed_file = chrt.fit_cosinor_fixed_period(animals[animal]['protocol'], best_models, save_folder = save_folder)    
    animals[animal]['best_models'] = best_models
    animals[animal]['best_models_file'] = best_models_file
    animals[animal]['best_models_fixed'] = best_models_fixed
    animals[animal]['best_models_fixed_file'] = best_models_fixed_file     

    end = time.time() - init

    print("Cosinor fitted to " + animal.replace('_', ' ') + " and results saved!")
    print("Time elapsed: " + "{:.2f}".format(end) + " seconds")

Cosinor fitted to animal 17 and results saved!
Time elapsed: 72.97 seconds
Cosinor fitted to animal 19 and results saved!
Time elapsed: 62.12 seconds


In [12]:
for count, animal in enumerate(animals):
    init = time.time()
    
    save_folder = animals[animal]['save_folder']

    chp.model_overview_detailed(animals[animal]['protocol'], animals[animal]['best_models_fixed'], save_folder = save_folder, format = format, ticks_fontsize=[14,14], labels_fontsize=[18,16,16])
    chp.model_overview(animals[animal]['protocol'], animals[animal]['best_models'], save_folder = save_folder, format = format, ticks_fontsize=[14,14], labels_fontsize=[18,16,16])
    chp.model_over_signal(animals[animal]['protocol'], animals[animal]['best_models'], position = 'head', mv_avg_window = 1, save_folder = save_folder, format = format, ticks_fontsize=[14,14], labels_fontsize=[18,16,16])

    end = time.time() - init

    print("Cosinor overview " + animal.replace('_', ' ') + " saved!")
    print("Time elapsed: " + "{:.2f}".format(end) + " seconds")

Cosinor overview animal 17 saved!
Time elapsed: 3.29 seconds
Cosinor overview animal 19 saved!
Time elapsed: 3.15 seconds


In [13]:
dict = {'day_window': 3, 
        'step': 0.5, 
        'start_time': 22, 
        'end_time': 27, 
        'n_components': [1]}

best_models = []

for count, animal in enumerate(animals):
    init = time.time()

    save_folder = animals[animal]['save_folder']

    best_models_per_day, best_models_fixed_file = chrt.fit_cosinor_per_day(animals[animal]['protocol'], dict = dict, plot = True, save_folder = save_folder)    
    chp.model_per_day(animals[animal]['protocol'], best_models_per_day, dict['day_window'], save_folder = save_folder, save_suffix = '', format = format)
    animals[animal]['best_models_per_day'] = best_models_per_day
    animals[animal]['best_models_file_per_day'] = best_models_fixed_file

    end = time.time() - init

    print("Cosinor fitted to " + animal.replace('_', ' ') + " and results saved!")
    print("Time elapsed: " + "{:.2f}".format(end) + " seconds")

Cosinor fitted to animal 17 and results saved!
Time elapsed: 63.78 seconds
Cosinor fitted to animal 19 and results saved!
Time elapsed: 60.41 seconds
